In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 73.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 86.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 83.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 21.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps

### Review Data

In [ ]:
# This is a label from the annotation folder. This label is .xml format.
with open("/content/drive/MyDrive/project/xml/all_xml/195_1b1_Lr_sc_Litt3200_3.xml") as f:
    contents = f.read()
    print(contents)

<?xml version="1.0" ?>
<annotation>
	
    
	<folder>crack</folder>
	
    
	<filename>195_1b1_Lr_sc_Litt3200_3.png</filename>
	
    
	<path>/content/drive/MyDrive/project/crack/195_1b1_Lr_sc_Litt3200_3.png</path>
	
    
	<source>
		
        
		<database>Unknown</database>
		
    
	</source>
	
    
	<size>
		
        
		<width>224</width>
		
        
		<height>224</height>
		
        
		<depth>3</depth>
		
    
	</size>
	
    
	<segmented>0</segmented>
	
    
	<object>
		
        
		<name>crackles</name>
		
        
		<pose>Unspecified</pose>
		
        
		<truncated>0</truncated>
		
        
		<difficult>0</difficult>
		
        
		<bndbox>
			
            
			<xmin>74</xmin>
			
            
			<ymin>205</ymin>
			
            
			<xmax>149</xmax>
			
            
			<ymax>215</ymax>
			
        
		</bndbox>
		
    
	</object>
	

	<object>
		<name>wheezes</name>
		<bndbox>
			<xmin>74</xmin>
			<ymin>115</ymin>
			<xmax>149</xmax>
			<ymax>206</ymax>
		</bndbox>
	</object>
</annotation

### Preparing Data For Model(YoloV9)

#### Translate '.xml" format to ".txt" format

In [ ]:
# The functions are about translating label data.
def xml_to_yolo_bbox(bbox, w, h):
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2 ) / h

    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h

    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center, width, height
    w_half_len = (bbox[2] + w) / 2
    h_half_len = (bbox[3] + h) / 2

    xmin = int((bbox[0] + w) - w_half_len)
    ymin = int((bbox[1] + h) - h_half_len)
    xmax = int((bbox[0] + w) + w_half_len)
    ymax = int((bbox[1] + h) + h_half_len)

    return [xmin, ymin, xmax, ymax]

In [ ]:
# Create .txt label data.
classes = []

input_dir = "/content/drive/MyDrive/project/xml/all_xml"
output_dir = "/content/drive/MyDrive/project/model/YoloV9/labels"
image_dir = "/content/drive/MyDrive/project/original"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist")
        continue

    result = []

    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text

        if label not in classes:
            classes.append(label)

        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)

        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:

        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))

with open(f"{output_dir}/classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))

In [ ]:
# Labels
with open(f"{output_dir}/classes.txt") as f:
    contents = f.read()
    print(contents)

["crackles", "wheezes"]


In [ ]:
# .xml format
with open("/kaggle/input/face-mask-detection/annotations/maksssksksss334.xml") as f:
    contents = f.read()
    print(contents)

In [ ]:
# .txt format
with open("/kaggle/working/labels/maksssksksss334.txt") as f:
    contents = f.read()
    print(contents)

In [ ]:
annotation_count = len(os.listdir("/content/drive/MyDrive/project/xml/all_xml"))
labels_count = len(os.listdir("/content/drive/MyDrive/project/model/YoloV9/labels"))
print(f"Annotation Counts: {annotation_count}")
print(f"Lables Counts: {labels_count}")

Annotation Counts: 3246
Lables Counts: 3247


In [ ]:
annotation_list = os.listdir("/content/drive/MyDrive/project/xml/all_xml")
labels_list = os.listdir("/content/drive/MyDrive/project/model/YoloV9/labels")

In [ ]:
import os

# annotation 디렉토리에 있는 파일 목록 가져오기
file_list = os.listdir("/content/drive/MyDrive/project/xml/all_xml")

# 확장자를 제외한 파일명을 저장할 리스트 초기화
file_names_without_extension = []

# 파일 목록에서 확장자를 제외한 파일명만 추출하여 리스트에 추가
for file_name in file_list:
    name_without_extension = os.path.splitext(file_name)[0]
    file_names_without_extension.append(name_without_extension)

# 확장자를 제외한 파일명이 저장된 리스트 출력
print("확장자를 제외한 파일명 리스트:", file_names_without_extension)

확장자를 제외한 파일명 리스트: ['130_1p2_Ll_mc_AKGC417L_6', '186_2b4_Pr_mc_AKGC417L_4', '186_3b3_Ar_mc_AKGC417L_2', '186_2b4_Pl_mc_AKGC417L_3', '186_2b4_Pr_mc_AKGC417L_0', '186_2b4_Pr_mc_AKGC417L_3', '186_2b4_Al_mc_AKGC417L_2', '186_3b3_Ar_mc_AKGC417L_4', '186_2b4_Al_mc_AKGC417L_3', '186_2b4_Pr_mc_AKGC417L_2', '186_2b4_Pr_mc_AKGC417L_1', '186_2b4_Pl_mc_AKGC417L_4', '186_2b4_Al_mc_AKGC417L_4', '186_2b4_Al_mc_AKGC417L_0', '186_2b4_Pl_mc_AKGC417L_2', '186_2b4_Pl_mc_AKGC417L_0', '205_4b2_Al_mc_AKGC417L_2', '207_2b4_Al_mc_AKGC417L_0', '205_4b2_Pr_mc_AKGC417L_4', '207_2b2_Pl_mc_AKGC417L_1', '205_4b2_Pr_mc_AKGC417L_3', '205_4b2_Lr_mc_AKGC417L_0', '205_4b2_Al_mc_AKGC417L_3', '207_2b3_Al_mc_AKGC417L_1', '205_4b2_Pr_mc_AKGC417L_1', '207_2b4_Al_mc_AKGC417L_1', '207_2b3_Al_mc_AKGC417L_2', '205_4b2_Al_mc_AKGC417L_0', '205_4b2_Al_mc_AKGC417L_1', '207_2b2_Pl_mc_AKGC417L_3', '205_4b2_Al_mc_AKGC417L_4', '198_6p1_Ar_mc_AKGC417L_5', '198_6p1_Ar_mc_AKGC417L_0', '198_6p1_Lr_mc_AKGC417L_0', '198_6p1_Pr_mc_AKGC417L_0', '

In [ ]:
# 디렉토리 경로
directory = "/content/drive/MyDrive/project/model/YoloV9/labels"

# 디렉토리 내의 파일 목록 가져오기
file_list = os.listdir(directory)

# .txt 확장자를 제외한 파일명 리스트 초기화
file_names_without_extension1 = []

# 파일 목록에서 .txt 확장자를 제외한 파일명만 추출하여 리스트에 추가
for file_name in file_list:
    name_without_extension = os.path.splitext(file_name)[0]
    file_names_without_extension1.append(name_without_extension)

# .txt 확장자를 제외한 파일명이 저장된 리스트 출력
print("확장자를 제외한 파일명 리스트:", file_names_without_extension1)

확장자를 제외한 파일명 리스트: ['130_1p2_Ll_mc_AKGC417L_6', '186_2b4_Pr_mc_AKGC417L_4', '186_3b3_Ar_mc_AKGC417L_2', '186_2b4_Pl_mc_AKGC417L_3', '186_2b4_Pr_mc_AKGC417L_0', '186_2b4_Pr_mc_AKGC417L_3', '186_2b4_Al_mc_AKGC417L_2', '186_3b3_Ar_mc_AKGC417L_4', '186_2b4_Al_mc_AKGC417L_3', '186_2b4_Pr_mc_AKGC417L_2', '186_2b4_Pr_mc_AKGC417L_1', '186_2b4_Pl_mc_AKGC417L_4', '186_2b4_Al_mc_AKGC417L_4', '186_2b4_Al_mc_AKGC417L_0', '186_2b4_Pl_mc_AKGC417L_2', '186_2b4_Pl_mc_AKGC417L_0', '205_4b2_Al_mc_AKGC417L_2', '207_2b4_Al_mc_AKGC417L_0', '205_4b2_Pr_mc_AKGC417L_4', '207_2b2_Pl_mc_AKGC417L_1', '205_4b2_Pr_mc_AKGC417L_3', '205_4b2_Lr_mc_AKGC417L_0', '205_4b2_Al_mc_AKGC417L_3', '207_2b3_Al_mc_AKGC417L_1', '205_4b2_Pr_mc_AKGC417L_1', '207_2b4_Al_mc_AKGC417L_1', '207_2b3_Al_mc_AKGC417L_2', '205_4b2_Al_mc_AKGC417L_0', '205_4b2_Al_mc_AKGC417L_1', '207_2b2_Pl_mc_AKGC417L_3', '205_4b2_Al_mc_AKGC417L_4', '198_6p1_Ar_mc_AKGC417L_5', '198_6p1_Ar_mc_AKGC417L_0', '198_6p1_Lr_mc_AKGC417L_0', '198_6p1_Pr_mc_AKGC417L_0', '

In [ ]:
# 차이를 찾기 위해 두 리스트를 세트(set)로 변환하여 차집합 연산을 수행
difference = set(file_names_without_extension1) - set(file_names_without_extension)

# 차이가 있는 파일 목록 출력
print("차이나는 파일 목록:", difference)

차이나는 파일 목록: {'classes'}


### Preparing Training Data

In [ ]:
# Create a folder of train dataset
if not os.path.isfile("/content/drive/MyDrive/project/model/YoloV9/Training/data"):
    # os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/train')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/val')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/test')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/train/images')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/train/labels')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/test/images')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/test/labels')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/val/images')
    os.mkdir('/content/drive/MyDrive/project/model/YoloV9/Training/data/val/labels')

In [ ]:
# Material is the names of images in the images folder.
metarial = []
for i in os.listdir("/content/drive/MyDrive/project/original"):
    str1 = i[:-4]
    metarial.append(str1)

In [ ]:
metarial[0:13]

['130_2p5_Pl_mc_AKGC417L_10',
 '130_3b3_Ll_mc_AKGC417L_0',
 '130_3b3_Ll_mc_AKGC417L_1',
 '130_3b3_Ll_mc_AKGC417L_2',
 '130_3b3_Ll_mc_AKGC417L_3',
 '130_3b3_Ll_mc_AKGC417L_4',
 '130_2p5_Pr_mc_AKGC417L_0',
 '130_2p5_Pr_mc_AKGC417L_1',
 '130_2p5_Pr_mc_AKGC417L_2',
 '130_2p5_Pr_mc_AKGC417L_3',
 '130_2p5_Pr_mc_AKGC417L_4',
 '130_2p5_Pr_mc_AKGC417L_5',
 '130_2p5_Pr_mc_AKGC417L_6']

In [ ]:
print("Sum of image: ", len(metarial))
train_size = int(len(metarial) * 0.7)
test_size = int(len(metarial) * 0.15)
val_size = int(len(metarial) * 0.15)
print("Sum of the train size: ", train_size)
print("Sum of the test size: ", test_size)
print("Sum of the val size: ", val_size)

Sum of image:  3246
Sum of the train size:  2272
Sum of the test size:  486
Sum of the val size:  486


In [ ]:
import os
import shutil

def preparingdata(main_txt_file, main_img_file, train_size, test_size, val_size):
    for i in range(0, train_size):
        source_txt = main_txt_file + "/" + metarial[i] + ".txt"
        source_img = main_img_file + "/" + metarial[i] + ".png"

        mstring = metarial[i]
        train_destination_txt = "/content/drive/MyDrive/project/model/YoloV9/Training/data/train/labels" + "/" + metarial[i] + ".txt"
        train_destination_png = "/content/drive/MyDrive/project/model/YoloV9/Training/data/train/images" + "/" + metarial[i] + ".png"

        shutil.copy(source_txt, train_destination_txt)
        shutil.copy(source_img, train_destination_png)

    for l in range(train_size , train_size + test_size):
        source_txt = main_txt_file + "/" + metarial[l] + ".txt"
        source_img = main_img_file + "/" + metarial[l] + ".png"

        mstring = metarial[l]
        test_destination_txt = "/content/drive/MyDrive/project/model/YoloV9/Training/data/test/labels" + "/" + metarial[l] + ".txt"
        test_destination_png = "/content/drive/MyDrive/project/model/YoloV9/Training/data/test/images" + "/" + metarial[l] + ".png"

        shutil.copy(source_txt, test_destination_txt)
        shutil.copy(source_img, test_destination_png)

    for n in range(train_size + test_size , train_size + test_size + val_size):
        source_txt = main_txt_file + "/" + metarial[n] + ".txt"
        source_img = main_img_file + "/" + metarial[n] + ".png"

        mstring = metarial[n]
        val_destination_txt = "/content/drive/MyDrive/project/model/YoloV9/Training/data/val/labels" + "/" + metarial[n] + ".txt"
        val_destination_png = "/content/drive/MyDrive/project/model/YoloV9/Training/data/val/images" + "/" + metarial[n] + ".png"

        # Remove _mel_spec from the filename
        new_filename = metarial[n]

        # Copy files with modified filename
        shutil.copy(source_txt, val_destination_txt)
        shutil.copy(source_img, val_destination_png.replace(metarial[n] , new_filename))


In [ ]:
#preparinbdata("/kaggle/working/labels", "/kaggle/input/face-mask-detection/images", 603, 150, 100)
preparingdata(main_txt_file = "/content/drive/MyDrive/project/model/YoloV9/labels",
              main_img_file = "/content/drive/MyDrive/project/original",
              train_size = train_size,
              test_size = test_size,
              val_size = val_size)


In [18]:
train_data = len(os.listdir("/content/drive/MyDrive/project/model/YoloV9/Training/data/train/images"))
vol_data = len(os.listdir("/content/drive/MyDrive/project/model/YoloV9/Training/data/val/images"))
test_data = len(os.listdir("/content/drive/MyDrive/project/model/YoloV9/Training/data/test/images"))

print(f"train_data: {train_data}")
print(f"vol_data: {vol_data}")
print(f"test_data: {test_data}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/project/model/YoloV9/Training/data/train/images'

### yaml 파일 제작

In [26]:
# Create the data.yaml. I am going to use it on the train with yolov9.
yaml_text = """train: /home/sogaksa123/AIFFEL_THON/model/Yolov9/Training/data/Training/data/train/images
val: /home/sogaksa123/AIFFEL_THON//model/Yolov9/Training/data/Training/data/val/images

nc: 2
names: ["crackles", "wheezes"]"""

with open("/home/sogaksa123/AIFFEL_THON/model/Yolov9/data.yaml", 'w') as file:
    file.write(yaml_text)

In [27]:
with open("/home/sogaksa123/AIFFEL_THON/model/Yolov9/data.yaml") as f:
    contents = f.read()
    print(contents)

train: /home/sogaksa123/AIFFEL_THON/model/Yolov9/Training/data/Training/data/train/images
val: /home/sogaksa123/AIFFEL_THON//model/Yolov9/Training/data/Training/data/val/images

nc: 2
names: ["crackles", "wheezes"]


### YoloV9

In [22]:
# Download Github rep.
!git clone https://github.com/SkalskiP/yolov9.git

fatal: destination path 'yolov9' already exists and is not an empty directory.


In [23]:
import os
current_directory = os.getcwd()
print(current_directory)

/home/sogaksa123/AIFFEL_THON/model/Yolov9


In [24]:
# İnstall the req.
!pip install -r /home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/requirements.txt -q

In [29]:
# Train with YoloV9
"""
workers --> max dataloader workers (per RANK in DDP mode)
device --> cuda device, i.e. 0 or 0,1,2,3 or cpu
batch --> total batch size for all GPUs, -1 for autobatch
epochs --> total training epochs
data --> dataset.yaml path
img --> train, val image size (pixels)
cfg --> model.yaml path
weights --> initial weights path
name --> save to project/name
hyp --> hyperparameters path

**This parameters is my project parameters. You can make changes for your project.**
"""

!python /home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/train_dual.py \
--workers 8 --device 0  --batch 32 --epochs 20 --min-items 0 --close-mosaic 15\
--data /home/sogaksa123/AIFFEL_THON/model/Yolov9/data.yaml \
--img 224 --cfg  /home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/models/detect/yolov9-c.yaml \
--weights '' --name yolov9-c --hyp /home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/data/hyps/hyp.scratch-high.yaml

train_dual: weights=, cfg=/home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/models/detect/yolov9-c.yaml, data=/home/sogaksa123/AIFFEL_THON/model/Yolov9/data.yaml, hyp=/home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/data/hyps/hyp.scratch-high.yaml, epochs=20, batch_size=32, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov9/runs/train, name=yolov9-c, exist_ok=False, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=15, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5 🚀 1e33dbb Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22491MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, w

##
Class: 클래스 레이블입니다.  
Images: 해당 클래스에 대한 이미지 수입니다.  
Instances: 해당 클래스에 대한 감지된 객체의 총 수입니다.  
P: 정밀도(Precision)는 모델이 예측한 객체 중 실제로 해당 클래스에 속하는 객체의 비율입니다. 높은 정밀도는 모델이 정확하게 예측한 것을 의미합니다.  
R: 재현율(Recall)은 실제 해당 클래스에 속하는 객체 중 모델이 정확하게 감지한 객체의 비율입니다. 높은 재현율은 모델이 대상을 놓치지 않고 잘 감지한다는 것을 의미합니다.  
mAP50: 평균 정밀도(mean Average Precision)는 다양한 IoU(Intersection over Union) 임계값에 대해 계산된 정밀도의 평균입니다. 이 경우, IoU 임계값이 0.5인 경우의 mAP입니다. 높은 mAP는 모델의 성능이 좋다는 것을 의미합니다.

해석:  
전체적으로 모델의 성능은 중간 정도로 보입니다. 정밀도와 재현율이 비슷한 수준이며, 평균 정밀도도 0.3 정도로 나쁘지 않은 편입니다.
클래스 별로 살펴보면, crackles와 wheezes 클래스의 재현율이 상대적으로 높지만 정밀도는 낮습니다. 이는 모델이 해당 클래스의 객체를 많이 감지하지만 그 중에서 실제 해당 클래스에 속하는 객체의 비율이 낮다는 것을 의미합니다.
개선을 위해서는 모델의 학습 데이터의 다양성을 높이거나, 모델 아키텍처를 조정하여 성능을 향상시킬 수 있습니다.

In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

# YOLOv5 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# 이미지 로드
image_path = 'your_image.jpg'  # 이미지 경로
image = Image.open(image_path)

# 객체 감지
results = model(image)

# 결과 시각화
results.show()

# 결과를 이미지로 저장
results.save('result.jpg')

# 결과를 numpy 배열로 반환
result_image = results.render()[0]
result_image = cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR)
cv2.imwrite('result_cv2.jpg', result_image)

# 결과를 matplotlib으로 시각화
plt.figure(figsize=(10, 10))
plt.imshow(result_image)
plt.axis('off')
plt.show()

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/home/sogaksa123/AIFFEL_THON/model/Yolov9


In [6]:
%cd '/home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/'

/home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9


In [ ]:
AIFFEL_THON/model/Yolov9/yolov9/detect.py

In [ ]:
# evaluate converted yolov9 models
python val.py --data data/coco.yaml --img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 --weights './yolov9-c-converted.pt' --save-json --name yolov9_c_c_640_val

In [10]:
!python detect.py --source '/home/sogaksa123/AIFFEL_THON/model/Yolov9/Training/data/Training/data/test/images/177_1b2_Pl_mc_AKGC417L_0.png' --img 224 --device 0 --weights '/home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/runs/train/yolov9-c5/weights/best.pt' --name yolov9_c_c_640_detect

detect: weights=['/home/sogaksa123/AIFFEL_THON/model/Yolov9/yolov9/runs/train/yolov9-c5/weights/best.pt'], source=/home/sogaksa123/AIFFEL_THON/model/Yolov9/Training/data/Training/data/test/images/177_1b2_Pl_mc_AKGC417L_0.png, data=data/coco128.yaml, imgsz=[224, 224], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=yolov9_c_c_640_detect, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 1e33dbb Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22491MiB)

Fusing layers... 
yolov9-c summary: 724 layers, 50960940 parameters, 0 gradients, 237.7 GFLOPs
image 1/1 /home/sogaksa123/AIFFEL_THON/model/Yolov9/Training/data/Training/data/test/images/177_1b2_Pl_mc_AKGC417L_0.png: 224x224 1 wheezes, 26.5ms
Speed: 0.2ms pre-process, 26.5ms infe